# Writing New Code with Copilot

We can use Copilot to write new pieces of code. When doing so, the following are useful tips:

1. **Provide a description**: Copilot can generate code based on the description you provide. The more detailed the description, the better the code generated.
2. **Be specific**: For example, when writing a function, you might specify:
    - The types and purposes of arguments
    - The return type
    - The expected behavior
    - Any constraints or requirements
    - If and when exceptions should be raised
3. **Iterate**: Copilot may not get it right the first time. Describe problems and what you would like done differently. This can be done in chat, or by make iterative changes in the editor.
4. **Set the style**: Copilot is aware of other code in the file and can generate code that is consistent with it. Writing some code before asking Copilot to generate more can help it understand the style you are looking for.
5. **Be critical of suggestions**: Copilot may generate code that is not correct or not what you want. It is important to review the code and make sure it is correct and does what you want.
6. **Test the code**: You should always test code you write, preferably by writing formal tests in a unit-testing framework such as ```unittest``` or ```pytest``` in Python. This is doubly important when using Copilot, as the code may not be correct or may not do what you expect. Copilot can help you write tests.
7. **Understand the code**: Copilot cannot replace the need for you to understand the code you're writing. It is best suited to speeding up the production of tasks you could do yourself. It may be able to help you produce code one step more complicated than you could do yourself, but you should make sure you understand what is going on before you use it. This can be a good way to learn new techniques or patterns, and can be aided by asking Copilot for explanations of bits you don't understand. If you try to produce code that is substantially beyond your current abilities, you will likely make mistakes and not understand how to fix them. As a result, you should carry on developing your skills as a programmer instead of relying on Copilot (or any other AI tool) to do the work for you.
8. **Break down large tasks**: Copilot is better at generating small pieces of code than large ones. If you have a large task, try to break it down into smaller pieces that you can ask Copilot to generate for you. This will make it easier for Copilot to understand what you want and generate the correct code. Copilot can help you with this process by suggesting ways to break down the task.


## Example

In the code cell below, the tutor will demonstrate how to write a function which accepts a number of data series in a Numpy array, and produces a line plot in Matplotlib.

## Exercise

Set yourself the task of writing a function that you could write yourself in 30-60 minutes. See if you can get Copilot to generate the code for you. While doing this be mindful of the following questions:
- Be mindful of what you are asking Copilot to do. What worked well? What didn't?
- Is the code correct?
- Is it different from what you would have written? Is it better of worse?
- Did you learn any new programming techniques?


In [7]:
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt
from torch.optim import LBFGS

# Define the function that takes a 5D input and computes sin+cos
def target_function(x):
    """
    Takes a 5D input tensor and computes sin+cos
    x: tensor of shape (..., 5)
    returns: tensor of shape (...)
    """
    return torch.sum(torch.sin(x) + torch.cos(x), dim=-1)

# Generate training data
def generate_data(n_samples=100, n_dims=5, test=False):
    """Generate random points in the 5D space"""
    if test:
        x = torch.rand(n_samples, n_dims) * 8.0 - 4.0  # Range [-4, 4]
    else:
        x = torch.rand(n_samples, n_dims) * 6.0 - 3.0  # Range [-3, 3]
    
    y = target_function(x)
    return x, y

# Define the GP model
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=5)
        )
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Train and evaluate the GP model
def train_and_evaluate_gp(n_train=100, n_test=50):
    # Generate training data
    train_x, train_y = generate_data(n_samples=n_train)
    
    # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    
    # Set model and likelihood in training mode
    model.train()
    likelihood.train()
    
    # Use the LBFGS optimizer
    optimizer = LBFGS(model.parameters(), lr=0.001, max_iter=10000)
    
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    # Training loop
    def closure():
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        return loss
    
    print("Training the GP model...")
    losses = []
    
    # Train for 50 iterations
    # for i in range(50):
    loss = optimizer.step(closure)
    losses.append(loss.item())
    print(f"Final loss: {loss.item()}")
    # if i % 10 == 0:
    #     print(f"Iteration {i}/{50}: Loss = {loss.item()}")
    
    print(f"Final loss: {losses[-1]}")
    
    # Generate test data
    test_x, test_y = generate_data(n_samples=n_test, test=True)
    
    # Evaluate the model
    model.eval()
    likelihood.eval()
    
    # Make predictions
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = likelihood(model(test_x))
        pred_mean = observed_pred.mean
    
    # Calculate RMSE
    rmse = torch.sqrt(torch.mean((pred_mean - test_y) ** 2))
    print(f"Test RMSE: {rmse.item()}")
    
    return model, rmse

# Run the training and evaluation
model, rmse = train_and_evaluate_gp(n_train=1000, n_test=100)

Training the GP model...


/home/choudhury/miniconda3/envs/post_env/lib/python3.12/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 1.7130115032196045 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/choudhury/miniconda3/envs/post_env/lib/python3.12/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 1.5806324481964111 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/choudhury/miniconda3/envs/post_env/lib/p

KeyboardInterrupt: 